In [41]:
from preprocess_utils import *

train, test = get_processed_data()
train = select_columns(train)
train.head()

/tmp/ipykernel_6653/459611240.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  train, test = get_processed_data()


,author,hindex,nb_paper,pagerank,authority,clustering_coef,n_neighbors_dist_1,mean_neighbors_dist_1,max_neighbors_dist_1,vector_coord_0,vector_coord_1
0,1964267543,4.0,5,0.000004,3.339138e-13,1.000000,5,21.800000,39,-0.186604,0.288065
1,2153592714,13.0,5,0.000002,1.433035e-17,1.000000,2,20.000000,20,-0.239073,0.446518
2,217158525,8.0,5,0.000003,6.750288e-20,1.000000,2,2.000000,2,-0.225883,0.357737
3,2123103677,11.0,3,0.000005,7.714573e-16,0.714286,7,11.833333,54,-0.174987,0.208499
4,2067710487,3.0,2,0.000004,-1.444459e-26,0.000000,2,9.841160,12,-0.170414,0.257662


In [35]:
frequency = train["hindex"].value_counts(normalize=True)

total = 0
for hindex, freq in frequency.items():
    total += freq
    if total > 0.95:
        print(hindex+1)
        break

In [10]:
large_hindex = 6

# Large Index Classification

In [35]:
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

train_split.loc[train_split["hindex"] < large_hindex, "hindex"] = 0
test_split.loc[test_split["hindex"] < large_hindex, "hindex"] = 0
train_split.loc[train_split["hindex"] >= large_hindex, "hindex"] = 1
test_split.loc[test_split["hindex"] >= large_hindex, "hindex"] = 1

print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

(130680, 11) (43561, 11)
(130680, 11)


In [36]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print(accuracy_score(y_test,mod_preds))
print(f1_score(y_test,mod_preds))
print(recall_score(y_test,mod_preds))

0.8647643534354124
0.8662686431636059
0.8677065805630088


In [40]:
probas = model_cat.predict_proba(X_test)

0.0 0.0 [0.99824989 0.00175011]
0.0 0.0 [0.98425203 0.01574797]
1.0 1.0 [0.01054319 0.98945681]
0.0 0.0 [0.95679013 0.04320987]
0.0 0.0 [0.87209915 0.12790085]
1.0 1.0 [0.02515714 0.97484286]
0.0 0.0 [9.99569386e-01 4.30613780e-04]
1.0 1.0 [0.08620745 0.91379255]
0.0 0.0 [0.91937371 0.08062629]
0.0 0.0 [0.93371935 0.06628065]
1.0 1.0 [0.07949701 0.92050299]
1.0 1.0 [0.1704983 0.8295017]
1.0 1.0 [0.26002652 0.73997348]
1.0 1.0 [0.17453031 0.82546969]
0.0 0.0 [0.59151858 0.40848142]
1.0 1.0 [0.20871168 0.79128832]
0.0 0.0 [0.99366963 0.00633037]
0.0 0.0 [0.99695208 0.00304792]
1.0 1.0 [0.28630761 0.71369239]
0.0 0.0 [0.95999832 0.04000168]
1.0 1.0 [0.13204153 0.86795847]
1.0 1.0 [0.0375949 0.9624051]
1.0 1.0 [1.89974905e-04 9.99810025e-01]
1.0 1.0 [3.49063773e-04 9.99650936e-01]
1.0 1.0 [0.38507063 0.61492937]
0.0 0.0 [0.82611982 0.17388018]
1.0 1.0 [0.00633185 0.99366815]
0.0 0.0 [0.99378894 0.00621106]
1.0 1.0 [8.66997542e-04 9.99133002e-01]
1.0 1.0 [0.27343202 0.72656798]
1.0 1.0 [0.0

In [13]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
model = HistGradientBoostingClassifier()
model.fit(X_train, y_train)
mod_preds = model.predict(X_test)
print(accuracy_score(y_test,mod_preds))
print(f1_score(y_test,mod_preds))
print(recall_score(y_test,mod_preds))

0.8662565138541356
0.8683806253388758
0.8735909090909091


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
model = KNeighborsClassifier()
model.fit(X_train, y_train)
mod_preds = model.predict(X_test)
print(accuracy_score(y_test,mod_preds))
print(f1_score(y_test,mod_preds))
print(recall_score(y_test,mod_preds))

0.7778287918091871
0.7828584249495175
0.793


In [22]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
model = LGBMClassifier()
model.fit(X_train, y_train)
mod_preds = model.predict(X_test)
print(accuracy_score(y_test,mod_preds))
print(f1_score(y_test,mod_preds))
print(recall_score(y_test,mod_preds))

0.8663712954248066
0.8683121055132005
0.8723181818181818


# Original Prediction MSE on large hindex

In [31]:
train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_big_test = [y for y in y_test if y >= large_hindex]
mod_big_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] >= large_hindex]
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_big_preds))

(130680, 11) (43561, 11)
Overall MSE: 48.50047673031719
Big Hindex MSE: 497.0707295639768


# Specific Prediction MSE on large hindex

In [40]:
big_train = train.loc[train["hindex"] >= large_hindex]

big_train_split, big_test_split = train_test_split(big_train)
print(big_train_split.shape, big_test_split.shape)

X_big_train = big_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_train = big_train_split["hindex"].to_numpy()
X_big_test = big_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_test = big_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_big_train, y_big_train)
mod_preds = model_cat.predict(X_big_test)
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_preds))

(6469, 11) (2157, 11)
Big Hindex MSE: 170.16960484061605


# Original Prediction MSE on small hindex

In [43]:
train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_small_test = [y for y in y_test if y < large_hindex]
mod_small_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] < large_hindex]
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_small_preds))

(130680, 11) (43561, 11)
Overall MSE: 51.41873596298948
Small Hindex MSE: 25.906271543186012


# Specific Prediction MSE on small hindex

In [44]:
small_train = train.loc[train["hindex"] < large_hindex]

small_train_split, small_test_split = train_test_split(small_train)
print(small_train_split.shape, small_test_split.shape)

X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_small_train, y_small_train)
mod_preds = model_cat.predict(X_small_test)
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_preds))

(124211, 11) (41404, 11)
Small Hindex MSE: 18.99344045082073


# Combine Specific prediction and Large Index Classification

In [66]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error

large_hindex = 6

train_split, test_split = train_test_split(train)
train_split_classifier, test_split_classifier = train_split.copy(), test_split.copy()
print(train_split.shape, test_split.shape)

large_train_split = train_split.loc[train_split["hindex"] >= large_hindex]
large_test_split = test_split.loc[test_split["hindex"] >= large_hindex]
small_train_split = train_split.loc[train_split["hindex"] < large_hindex]
small_test_split = test_split.loc[test_split["hindex"] < large_hindex]

train_split_classifier.loc[train_split_classifier["hindex"] < large_hindex, "hindex"] = 0
train_split_classifier.loc[train_split_classifier["hindex"] >= large_hindex, "hindex"] = 1
test_split_classifier.loc[test_split_classifier["hindex"] < large_hindex, "hindex"] = 0
test_split_classifier.loc[test_split_classifier["hindex"] >= large_hindex, "hindex"] = 1

# Balance small and large data
# large_train_split_temp = large_train_split.append(small_train_split.sample(n=int(len(small_train_split) / 2.5), ignore_index=True))
# small_train_split = small_train_split.append(large_train_split_temp.sample(n=int(len(large_train_split) / 1.5), ignore_index=True))
# large_train_split = large_train_split_temp

print(large_train_split.shape, small_train_split.shape)

# Data for the classifier
X_train_classifier = train_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train_classifier = train_split_classifier["hindex"].to_numpy()
X_test_classifier = test_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test_classifier = test_split_classifier["hindex"].to_numpy()

# Data for large hindex regressor
X_large_train = large_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_train = large_train_split["hindex"].to_numpy()
X_large_test = large_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_test = large_test_split["hindex"].to_numpy()

# Data for small hindex regressor
X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

# Original Data
X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()


model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Classical Hindex MSE:", mean_squared_error(y_test, preds))
print("Classical Hindex MSE on small:", mean_squared_error(y_small_test, model.predict(X_small_test)))
print("Classical Hindex MSE on large:", mean_squared_error(y_large_test, model.predict(X_large_test)))


model_small = CatBoostRegressor(verbose=False)
model_small.fit(X_small_train, y_small_train)
small_preds = model_small.predict(X_test)
print("Small Hindex MSE:", mean_squared_error(y_test, small_preds))
print("Small Hindex MSE on small:", mean_squared_error(y_small_test, model_small.predict(X_small_test)))
print("Small Hindex MSE on large:", mean_squared_error(y_large_test, model_small.predict(X_large_test)))


model_large = CatBoostRegressor(verbose=False)
model_large.fit(X_large_train, y_large_train)
large_preds = model_large.predict(X_test)
print("Large Hindex MSE:", mean_squared_error(y_test, large_preds))
print("Large Hindex MSE on small:", mean_squared_error(y_small_test, model_large.predict(X_small_test)))
print("Large Hindex MSE on large:", mean_squared_error(y_large_test, model_large.predict(X_large_test)))


hindex_classifier = CatBoostClassifier(verbose=False, num_trees=2000)
hindex_classifier.fit(X_train_classifier, y_train_classifier)
hindex_classifier_preds = hindex_classifier.predict(X_test_classifier)
print(accuracy_score(y_test_classifier, hindex_classifier_preds), f1_score(y_test_classifier,hindex_classifier_preds), recall_score(y_test_classifier,hindex_classifier_preds))

hindex_classifier_probas = hindex_classifier.predict_proba(X_test_classifier)

combine_preds = []
count_small, count_large, count_classic = 0, 0, 0
count_small_correct, count_large_correct = 0, 0

threshold = 0.95
for i in range(len(y_test)):
    # if hindex_classifier_preds == 0:
    #     combine_preds.append(small_preds[i])
    # else:
    #     combine_preds.append(large_preds[i])
    if hindex_classifier_probas[i][0] > threshold:
        combine_preds.append(small_preds[i])
        count_small += 1
        if y_test_classifier[i] == 0:
            count_small_correct += 1
    elif hindex_classifier_probas[i][1] > threshold:
        combine_preds.append(large_preds[i])
        count_large += 1
        if y_test_classifier[i] == 1:
            count_large_correct += 1
    else:
        combine_preds.append(preds[i])
        count_classic += 1
print("Combine Hindex MSE:", mean_squared_error(y_test, combine_preds))
print("count :", count_small, count_large, count_classic)
print("count correct:", count_small_correct, count_large_correct)
print("ratio correct:", count_small_correct/count_small, count_large_correct/count_large)

(130680, 11) (43561, 11)
(91857, 11) (108688, 11)
Classical Hindex MSE: 50.489632485124815
Classical Hindex MSE on small: 10.597173756539663
Classical Hindex MSE on large: 89.53940764905308
Small Hindex MSE: 52.29774599561658
Small Hindex MSE on small: 6.893957728274923
Small Hindex MSE on large: 96.74243002708334
Large Hindex MSE: 51.84898877701079
Large Hindex MSE on small: 16.189760796716545
Large Hindex MSE on large: 86.75495545667194
0.8669452032781617 0.8686369611531662 0.8705310498341889
Combine Hindex MSE: 50.39453416256308
count : 10055 10059 23447
count correct: 9910 9928
ratio correct: 0.9855793137742417 0.9869768366636843


# Duplicate very large hindex

In [79]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error

very_large_hindex = 36

train_split, test_split = train_test_split(train)

very_large_train_split = train_split.loc[train_split["hindex"] >= very_large_hindex]

train_split = train_split.append(very_large_train_split)
print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()


model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Classical Hindex MSE:", mean_squared_error(y_test, preds))


(130680, 11)
Classical Hindex MSE: 51.031006030063494


# Fasttext Classification

In [3]:
import numpy as np
from preprocess_utils import df_to_txt

large_hindex = 6
train_fasttext, test_fasttext = get_processed_data()
train_fasttext["hindex_lab"] = np.where(train_fasttext["hindex"] >= large_hindex, "__label__1", "__label__0")

train_fasttext_split, test_fasttext_split = train_test_split(train_fasttext)

path_fasttext_text = "../tmp/fasttext_text.txt"
df_to_txt(train_fasttext, path_fasttext_text)
model_fasttext = fasttext.train_supervised(
        path_fasttext_text, lr=0.15815, dim=2, epoch=33, wordNgrams=3
)
os.remove(path_fasttext_text)

/tmp/ipykernel_6653/626770302.py:5: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  train_fasttext, test_fasttext = get_processed_data()
Read 99M words
Number of words:  359997
Number of labels: 2
Progress: 100.0% words/sec/thread: 5831801 lr:  0.000000 avg.loss:  0.556754 ETA:   0h 0m 0s


In [4]:
path_fasttext_text = "../tmp/fasttext_text.txt"
df_to_txt(test_fasttext_split, path_fasttext_text)
print(model_fasttext.test(path_fasttext_text))
os.remove(path_fasttext_text)

(43561, 0.7491104428272997, 0.7491104428272997)


## Add Fasttext prediction to data

In [33]:
preds = model_fasttext.predict(train_fasttext["text"])

AttributeError: 'Series' object has no attribute 'find'

In [19]:
train.head()

,author,hindex,nb_paper,pagerank,authority,clustering_coef,n_neighbors_dist_1,mean_neighbors_dist_1,max_neighbors_dist_1,vector_coord_0,vector_coord_1,fasttext_pred
0,1964267543,4.0,5,0.000004,3.339138e-13,1.000000,5,21.800000,39,-0.186604,0.288065,__label__1
1,2153592714,13.0,5,0.000002,1.433035e-17,1.000000,2,20.000000,20,-0.239073,0.446518,__label__0
2,217158525,8.0,5,0.000003,6.750288e-20,1.000000,2,2.000000,2,-0.225883,0.357737,__label__1
3,2123103677,11.0,3,0.000005,7.714573e-16,0.714286,7,11.833333,54,-0.174987,0.208499,__label__1
4,2067710487,3.0,2,0.000004,-1.444459e-26,0.000000,2,9.841160,12,-0.170414,0.257662,__label__1


In [28]:
from tqdm import tqdm
import pandas as pd

fasttext_pred = []
for text in tqdm(train_fasttext["text"].to_list()):
    if not pd.isna(text):
        label, proba = model_fasttext.predict(text)
    else:
        label, proba = model_fasttext.predict("")
    if label[0] == "__label__1":
        fasttext_pred.append(proba[0])
    else: 
        fasttext_pred.append(1-proba[0])

100%|██████████| 174241/174241 [00:14<00:00, 11636.55it/s]


In [29]:
train["fasttext_pred"] = fasttext_pred